In [ ]:
#| default_exp download

# download

> Download new data from internet

In [ ]:
#| export
import pathlib
import urllib
import datetime

import urllib3
import pandas as pd
# import rich.progress
# from tqdm.autonotebook import tqdm
from tqdm import tqdm

import sproc.structure

The *base* URLs and file names for every kind of file hosted (the final `/` at the end of each `base_url` is required for `urllib.parse.urljoin` below to work correctly).

In [ ]:
outsiders_info = {'base_url': 'https://contrataciondelsectorpublico.gob.es/sindicacion/sindicacion_1044/', 'base_filename': 'PlataformasAgregadasSinMenores_'}
insiders_info = {'base_url': 'https://contrataciondelsectorpublico.gob.es/sindicacion/sindicacion_643/', 'base_filename': 'licitacionesPerfilesContratanteCompleto3_'}
minors_info = {'base_url': 'https://contrataciondelsectorpublico.gob.es/sindicacion/sindicacion_1143/', 'base_filename': 'contratosMenoresPerfilesContratantes_'}

A function to download a file from internet.

In [ ]:
#| export
def file(
    url: str, # URL for the file to be downloaded
    output_file: str | pathlib.Path # Name of the local file to be saved
    ) -> None:

    # in case a `str` was passed
    output_file = pathlib.Path(output_file)

    pool_manager = urllib3.PoolManager()

    try:
    
        # the request is made
        request = pool_manager.request('GET', url)

    except urllib3.exceptions.MaxRetryError:

        print(f'can\'t download "{url}"')

    with output_file.open('wb') as f:

        f.write(request.data)

A sample file (from this repository) is downloaded

In [ ]:
url = 'https://raw.githubusercontent.com/manuvazquez/sproc/main/samples/2018-2021_20samples.parquet'
output_file = 'download_sample.parquet'
file(url, output_file)

Let us check it is readable

In [ ]:
pd.read_parquet(output_file).head()

id  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
zip      file name                                          entry                                                      
some.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    https://contrataciondelestado.es/sindicacion/P...   
                                                            452    https://contrataciondelestado.es/sindicacion/P...   
                                                            451    https://contrataciondelestado.es/sindicacion/P...   
                                                            450    https://contrataciondelestado.es/sindicacion/P...   
                                                            449    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                             summary  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
zip      file name                                          entry                                                      
some.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    Expediente: 1284/17, Entidad: Diputación Provi...   
                                                            452    Expediente: 1282/17, Entidad: Diputación Provi...   
                                                            451    Expediente: 1281/17, Entidad: Diputación Provi...   
                                                            450    Id licitación: VI/17/04-015; Órgano de Contrat...   
                                                            449    Id Licitación: PcPG/2017/194222, Órgano de Con...   

                                                                                                               title  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                

In [ ]:
aggregate_2018 = 'https://contrataciondelsectorpublico.gob.es/sindicacion/sindicacion_1044/PlataformasAgregadasSinMenores_2018.zip'

ULRs are produced from the given date, `from_date`, onwards.

In [ ]:
#| export
def make_urls(
    base_url: str, # URL to the server including the hosting directory
    base_filename: str, # File name without neither date information nor extension
    from_date: datetime.datetime # The starting date
    ) -> list[tuple[str, str]]: # List of tuples (URL, file name)
    "Assemble URLs for files of a given kind that are to be downloaded"

    # a "hack" to get a date one month (never mind the day) after `from_date`
    next_month = from_date.replace(day=28) + datetime.timedelta(days=5)

    today = datetime.datetime.today()

    end_year = today.year
    end_month = today.month

    urls_filenames = []

    def append(filename: str):

        urls_filenames.append((urllib.parse.urljoin(base_url, filename), filename))

    # if a month after `from_date` is still the same year...
    if next_month.year == from_date.year:
    
        for month in range(next_month.month, 12+1):

            filename = base_filename + str(next_month.year) + str(month).zfill(2) + '.zip'

            append(filename)

    for year in range(next_month.year+1, today.year):

        filename = base_filename + str(year) + '.zip'

        append(filename)
    
    for month in range(1, today.month):
        
        filename = base_filename + str(today.year) + str(month).zfill(2) + '.zip'

        append(filename)

    return urls_filenames

As an example, let us assemble the URLs of all the *outsider*s files from November 2019 on.

In [ ]:
urls_filenames = make_urls(**sproc.structure.tables['outsiders'], from_date=datetime.datetime(2019, 11, 1))
urls_filenames[:3]
# urls_filenames

[('https://contrataciondelsectorpublico.gob.es/sindicacion/sindicacion_1044/PlataformasAgregadasSinMenores_201912.zip',
  'PlataformasAgregadasSinMenores_201912.zip'),
 ('https://contrataciondelsectorpublico.gob.es/sindicacion/sindicacion_1044/PlataformasAgregadasSinMenores_2020.zip',
  'PlataformasAgregadasSinMenores_2020.zip'),
 ('https://contrataciondelsectorpublico.gob.es/sindicacion/sindicacion_1044/PlataformasAgregadasSinMenores_2021.zip',
  'PlataformasAgregadasSinMenores_2021.zip')]

In [ ]:
make_urls(**sproc.structure.tables['outsiders'], from_date=datetime.datetime(2022, 1, 1) - datetime.timedelta(days=1))

[]

In order to *actually* download all the files of a given `kind` from a certain `date.

In [ ]:
#| export
def from_date(
    kind: str, # One of 'outsiders', 'insiders', or 'minors'
    date: datetime.datetime, # The starting date
    output_directory: str | pathlib.Path = pathlib.Path.cwd() # Output directory, defaults is the current one
    ) -> list[pathlib.Path]: # File name of every downloaded file
    "Downloads all the files of a given kind from a certain moment in time"

    # in case a `str` was passed
    output_directory = pathlib.Path(output_directory)

    # `kind` should be one of the pre-set types
    assert kind in sproc.structure.tables

    info = sproc.structure.tables[kind]

    urls_filenames = make_urls(**info, from_date=date)

    every_output_file = []

    for url, filename in tqdm(urls_filenames, desc='Downloading raw data'):
    # for url, filename in rich.progress.track(urls_filenames, description='Downloading raw data'):

        output_file = output_directory / filename

        # file is annotated *regardless* of whether download was needed or it was already there
        every_output_file.append(output_file)

        if output_file.exists():

            # print(f'"{output_file.name}" already exists')
            tqdm.write(f'"{output_file.name}" already exists')
            
            continue

        # file is actually downloaded
        file(url, output_file)

    return every_output_file

Let us make a new directory...

In [ ]:
output_directory = pathlib.Path.cwd().parent / 'downloads'
output_directory.mkdir(exist_ok=True)
print(output_directory)

/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/downloads


...into which files are to be downloaded

In [ ]:
dl_files = from_date('outsiders', datetime.datetime(2021, 10, 1), output_directory=output_directory)

"PlataformasAgregadasSinMenores_202111.zip" already exists
"PlataformasAgregadasSinMenores_202112.zip" already exists
"PlataformasAgregadasSinMenores_2022.zip" already exists


In [ ]:
#| hide
from nbdev.doclinks import nbdev_export

In [ ]:
#| hide
nbdev_export('80_download.ipynb')